<a href="https://colab.research.google.com/github/aishafarooque/Committee-Based-Classifier/blob/main/Committee-Based-Classifier/MOT/FairMOT_MOT16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tracking Models to Run Inferences on the MOT-16 dataset

Multi-Object Tracking (MOT) describes the task of identifying all objects in an image and following them over a sequence of frames. 

A wide range of Deep Learning-based MOT methods has been developed for human detection, but not for sparse/crowded space detection.

### 1. Install Packages and Setup Environment

In [7]:
# Get the FairMOT Model
! git clone https://github.com/ifzhang/FairMOT     # the FairMot scripts
! git clone https://github.com/CharlesShang/DCNv2  # package for using DLA-34 model, the backbone neural network of FairMOT

fatal: destination path 'FairMOT' already exists and is not an empty directory.
fatal: destination path 'DCNv2' already exists and is not an empty directory.


#### Ensure CUDA is enabled

In [8]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


#### Ensure the Python version is 3.8

In [9]:
! python3 --version

Python 3.7.10


#### Install PyTorch

In [10]:
# Duplicate cell, torch is installed again down below
# ! pip install torch==1.4.0 torchvision==0.5.0

#### Install CONDA

If CONDA is already installed, skip this section.

In [11]:
! conda --version

conda 4.9.2


Install Conda which is required for Fair MOT's requirements.

In [12]:
! pip install -q condacolab

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


#### Build DCNv2 - Deformable Convolutional Networks v2 with Pytorch

Ensure GPU is enabled before running build. 

Debugging link: https://github.com/CharlesShang/DCNv2/issues/98#issuecomment-939261953

Remove the build folder if it exists

In [13]:
! rm -rf /content/build

Build DCNv2

In [14]:
! python ./DCNv2/setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcpu.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/dcn_v2.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/__init__.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcuda.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/setup.py -> build/lib.linux-x86_64-3.7/DCNv2
running build_ext
building '_ext' extension
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/DCNv2
creating build/temp.linux-x86_64-3.7/content/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cuda
g++ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DWITH_CUDA -I/content/DCNv2/src -I/usr/local/lib/python3.7/site-packages/torch/include -I/usr/local/lib/python3.7/site-packages/torch/include/tor

Run test CUDA.  

For the *UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 'At least one of the inputs that requires gradient '* error:

* These are known issues on DCNv2 and are related to shortcomings related to grad-check, it is working fine for me (Even the Jacobian mismatch error is occuring).

Source: https://github.com/Mukosame/Zooming-Slow-Mo-CVPR-2020/issues/11#issuecomment-611352001

In [15]:
! python /content/DCNv2/testcuda.py

torch.Size([2, 64, 128, 128])
torch.Size([20, 32, 7, 7])
torch.Size([20, 32, 7, 7])
torch.Size([20, 32, 7, 7])
0.971507, 1.943014
0.971507, 1.943014
Zero offset passed
/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py:242: UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 
  'At least one of the inputs that requires gradient '
check_gradient_dpooling: True
Traceback (most recent call last):
  File "/content/DCNv2/testcuda.py", line 265, in <module>
    check_gradient_dconv()
  File "/content/DCNv2/testcuda.py", line 97, in check_gradient_dconv
    eps=1e-3, atol=1e-4, rtol=1e-2))
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py", line 295, in gradcheck
    'The tolerance for nondeterminism was {}.'.format(nondet_tol))
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py", line 2

#### Install FairMOT requirements

In [16]:
! pip install -r /content/FairMOT/requirements.txt
! conda install pytorch==1.2.0 torchvision==0.4.0 cudatoolkit=10.0 -c pytorch -y
! conda install ffmpeg -y

     |████████████████████████████████| 60.5 MB 1.4 MB/s 
     |████████████████████████████████| 596 kB 646 kB/s 
     |████████████████████████████████| 41 kB 145 kB/s 
     |████████████████████████████████| 40 kB 3.7 MB/s 
ERROR: Could not find a version that satisfies the requirement cython-bbox
ERROR: No matching distribution found for cython-bbox
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
- \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=10.0
    - pytorch==1.2.0
    - torchvision==0.4.0


The following packages will be downloaded:

    pa

#### Install packages for the DLA-34 Model

If these packages were missing, make sure to restart your runtime after installing.

In [17]:
! pip install boto3
! pip install opencv-python
! pip install sagemaker

     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 8.6 MB 39.8 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 247 kB 51.6 MB/s 


  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
     |████████████████████████████████| 520 kB 5.3 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 1.0 MB 37.8 MB/s 
     |████████████████████████████████| 40 kB 5.0 MB/s 
     |████████████████████████████████| 11.3 MB 38.0 MB/s 
     |████████████████████████████████| 81 kB 10.5 MB/s 
     |████████████████████████████████| 98 kB 6.4 MB/s 
     |████████████████████████████████| 503 kB 38.4 MB/s 
     |████████████████████████████████| 65 kB 3.7 MB/s 
     |████████████████████████████████| 112 kB 41.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Created wheel for sagemaker: filename=sagemaker-2.82.2-py2.py3-none-any.whl size=719285 sha256=4fee4c2bd390cbac498dd4c900e975e370d377b22ae44a567094a172ed4da987
  Stored in directory: /root/.cache/pip/wheels/51/

#### Download pre-trained model weights of the DLA-34 backbone model

Documentation for AWS SageMaker: https://aws.amazon.com/sagemaker/



In [ ]:
# import os

# os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-1'

In [ ]:
# import boto3
# import cv2
# from PIL import Image
# import sagemaker

# sess = sagemaker.Session()
# s3 = boto3.client('s3')

# # S3 bucket name and directories for the pre-trained DLA34 model
# bucket = 'pedestrian-tracker'  
# s3key_model = 'raw-pretrained-model'
# fname = 'fairmot_dla34.pth'
# s3.download_file(bucket, s3key + '/' + fname,  '/content/FairMOT/model/' + fname)


In [18]:
!pip install matplotlib
!pip install ipykernel

     |████████████████████████████████| 11.2 MB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.5 MB/s 
     |████████████████████████████████| 899 kB 38.5 MB/s 


     |████████████████████████████████| 130 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 280 kB 50.8 MB/s 
     |████████████████████████████████| 102 kB 3.3 MB/s 
     |████████████████████████████████| 428 kB 51.5 MB/s 
     |████████████████████████████████| 130 kB 50.4 MB/s 
     |████████████████████████████████| 1.8 MB 43.5 MB/s 
     |████████████████████████████████| 793 kB 41.3 MB/s 
     |████████████████████████████████| 1.6 MB 39.8 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.3 MB/s 
     |████████████████████████████████| 380 kB 53.2 MB/s 
     |████████████████████████████████| 100 kB 10.7 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 49.6.0.post20210108
    Uninstalling setuptools-49.6.0.po

In [19]:
! sudo apt-get update
! sudo apt-get install libssl-dev
# ! pip install -e git+https://github.com/samson-wang/cython_bbox.git#egg=cython-bbox

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

#### Installing Cython BBox

1. Download the tar file from https://pypi.org/project/cython-bbox/#files

2. Make this change: extra_compile_args=['-Wno-cpp',  '-Wno-unused-function', '-std=c99'], in setup.py. (Source: https://github.com/Zhongdao/Towards-Realtime-MOT/issues/165#issuecomment-1042496502)

3. Zip up file again, "tar -cvzf cython_bbox-0.1.3.tar.gz cython_bbox-0.1.3".  https://www.makeuseof.com/extract-tar-gz/

4. Upload and install on Google Colab.


In [1]:
! sudo tar -xvf /content/cython_bbox-0.1.3.tar.gz
! pip install -e git+https://github.com/samson-wang/cython_bbox.git#egg=cython-bbox
! pip install -e /content/cython_bbox-0.1.3/dist/cython_bbox-0.1.3-py3.9-macosx-11-x86_64.egg
! pip install -e /path/cython_bbox-0.1.3.tar/dist/cython_bbox-0.1.3/cython_bbox-0.1.3
! python setup.py bdist_wheel
! pip install cython_bbox

tar: /content/cython_bbox-0.1.3.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Obtaining cython-bbox from git+https://github.com/samson-wang/cython_bbox.git#egg=cython-bbox
  Cloning https://github.com/samson-wang/cython_bbox.git to ./src/cython-bbox
  Running command git clone -q https://github.com/samson-wang/cython_bbox.git /content/src/cython-bbox
  Running setup.py develop for cython-bbox
ERROR: Command errored out with exit status 1: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/content/src/cython-bbox/setup.py'"'"'; __file__='"'"'/content/src/cython-bbox/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' develop --no-deps Check the logs for full command output.
ERROR: /content/cytho

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [17]:
# ! pip install motmetrics
# ! pip3 install dcnv2

# ! sudo apt-get install libxml2-dev libxmlsec1-dev
# ! pip install p5py
# ! pip install PEP517
# ! python -m pip install --upgrade pip
# ! python -m pip install --no-use-pep517 bcrypt
# ! git clone https://github.com/haruishi43/dcn_v2
# ! cd dcn_v2
# ! pip install -r /content/dcn_v2/requirements-dev.txt
# ! pip install /content/dcn_v2

! python -m pip install --upgrade pip
# ! python -m pip install --no-use-pep517 dcn_v2
! pip install DCNv2


# ! rm -rf dcn_v2

  Using cached DCNv2-0.4.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for DCNv2
  Running setup.py clean for DCNv2
Failed to build DCNv2
  error: subprocess-exited-with-error
  
  × Running setup.py install for DCNv2 did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for DCNv2 ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> DCNv2

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.


In [4]:
!python /content/FairMOT/src/demo.py mot --load_model /content/FairMOT/src/lib/models/model.py \
        --conf_thres 0.3 --det_thres 0.3 --nms_thres 0.4 --track_buffer 30 \
        --input-video /content/FairMOT/videos/MOT16-03.mp4 \
        --output-root /content/FairMOT/output

python3: can't open file '/content/FairMOT/src/demo.py': [Errno 2] No such file or directory
